Forward Transform

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import math


In [12]:
import os
from skimage.transform import radon, iradon, rescale
from skimage.io import imsave
from skimage import img_as_ubyte

#from skimage.data import shepp_logan_phantom
#image = shepp_logan_phantom()
#image = rescale(image, scale=0.4, mode='reflect', channel_axis=None)

image_dir = r"C:\Users\Christos\Desktop\CT_Reconstruction\dataset\CT_128\hr_128"
sinogram_dir = r"C:\Users\Christos\Desktop\CT_Reconstruction\dataset\CT_128\lr_128"
inverse_dir = r"C:\Users\Christos\Desktop\CT_Reconstruction\dataset\CT_128\sr_128_128"
os.makedirs(image_dir, exist_ok=True)
os.makedirs(sinogram_dir, exist_ok=True)
os.makedirs(inverse_dir, exist_ok=True)

def normalize_image(image):
    min_val = np.min(image)
    max_val = np.max(image)
    return (image - min_val) / (max_val - min_val)

def generate_images_and_sinograms(num_samples):
    for i in range(num_samples):

        xx = np.linspace(0, 1 , 128)
        yy = np.linspace(0, 1, 128)
        x,y = np.meshgrid(xx, yy)
        r = np.sqrt((x - 0.5)**2 + (y-0.5)**2)
        f = np.zeros([128, 128])
        nb = int(np.random.uniform(1, 10, 1))

        for j in range(0, nb):
            a = np.random.uniform(0, 1, 1)
            x0 = np.random.uniform(0, 1, 1)
            y0 = np.random.uniform(0, 1, 1)
            sx = np.random.uniform(0.01, 0.3, 1)
            sy = np.random.uniform(0.01, 0.3, 1)
            f += a*np.exp(-0.5*((x- x0)**2/sx**2 + (y-y0)**2/sy**2))
        f[r>0.5] = 0
 
        image = f
        image = normalize_image(image)
        image = img_as_ubyte(image)

        image_filename = os.path.join(image_dir, f"image_{i+1:03d}.png")
        imsave(image_filename, image)

        theta = np.linspace(0.0, 360.0, max(image.shape), endpoint=False)
        sinogram = radon(image, theta=theta)

        noise_std = np.random.uniform(0, 13)
        sinogram += np.random.normal(0, noise_std, np.shape(sinogram))

        sinogram = normalize_image(sinogram)
        sinogram = img_as_ubyte(sinogram)
       
        sinogram_filename = os.path.join(sinogram_dir, f"sinogram_{i+1:03d}.png")
        imsave(sinogram_filename, sinogram)

        '''
        mean_signal_level = np.mean(sinogram)
        percentage_noise = (noise_std / mean_signal_level) * 100

        print(f"Noise Standard Deviation: {noise_std:.2f}")
        print(f"Mean Signal Level: {mean_signal_level:.2f}")
        print(f"Percentage of Noise: {percentage_noise:.2f}%")
        '''
        
        inverse = iradon(sinogram, theta=theta)

        inverse = normalize_image(inverse)
        inverse = img_as_ubyte(inverse)

        inverse_filename = os.path.join(inverse_dir, f"inverse_{i+1:03d}.png")
        imsave(inverse_filename, inverse)

        '''
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))
        ax1.set_title("Original")
        ax1.imshow(image, cmap=plt.cm.Greys_r)
        dx, dy = 0.5 * 360.0 / max(image.shape), 0.5 / sinogram.shape[0]
        ax2.set_title("Radon transform\n(Sinogram)")
        ax2.set_xlabel("Projection angle (deg)")
        ax2.set_ylabel("Projection position (pixels)")
        ax2.imshow(
            sinogram,
            cmap=plt.cm.Greys_r,
            extent=(-dx, 360.0 + dx, -dy, sinogram.shape[0] + dy),
            aspect='auto',
        )
        fig.tight_layout()
        plt.show()
        '''

generate_images_and_sinograms(2000)

C:\Users\Christos\AppData\Local\Temp\ipykernel_16216\1572917398.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  nb = int(np.random.uniform(1, 10, 1))


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


FBP

In [ ]:
import matplotlib.pyplot as plt
from skimage.transform.radon_transform import _get_fourier_filter

filters = ['ramp', 'shepp-logan', 'cosine', 'hamming', 'hann']

for ix, f in enumerate(filters):
    response = _get_fourier_filter(2000, f)
    plt.plot(response, label=f)

plt.xlim([0, 1000])
plt.xlabel('frequency')
plt.legend()
plt.show()

In [ ]:
from skimage.transform import iradon

reconstruction_fbp = iradon(sinogram, theta=theta, filter_name='ramp')
error = reconstruction_fbp - image
print(f'FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')

imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)
ax1.set_title("Reconstruction\nFiltered back projection")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nFiltered back projection")
ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

SART

In [ ]:
from skimage.transform import iradon_sart

reconstruction_sart = iradon_sart(sinogram, theta=theta)
error = reconstruction_sart - image
print(
    f'SART (1 iteration) rms reconstruction error: ' f'{np.sqrt(np.mean(error**2)):.3g}'
)

fig, axes = plt.subplots(2, 2, figsize=(8, 8.5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].set_title("Reconstruction\nSART")
ax[0].imshow(reconstruction_sart, cmap=plt.cm.Greys_r)

ax[1].set_title("Reconstruction error\nSART")
ax[1].imshow(reconstruction_sart - image, cmap=plt.cm.Greys_r, **imkwargs)

reconstruction_sart2 = iradon_sart(sinogram, theta=theta, image=reconstruction_sart)
error = reconstruction_sart2 - image
print(
    f'SART (2 iterations) rms reconstruction error: '
    f'{np.sqrt(np.mean(error**2)):.3g}'
)

ax[2].set_title("Reconstruction\nSART, 2 iterations")
ax[2].imshow(reconstruction_sart2, cmap=plt.cm.Greys_r)

ax[3].set_title("Reconstruction error\nSART, 2 iterations")
ax[3].imshow(reconstruction_sart2 - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

In [19]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale

image_dir = r"C:\Users\Christos\Desktop\1"
os.makedirs(image_dir, exist_ok=True)

def generate_images_and_sinograms(num_samples):
    for i in range(num_samples):

        image = shepp_logan_phantom()
        image = rescale(image, scale=0.32, mode='reflect', channel_axis=None)
        image = normalize_image(image)
        image = img_as_ubyte(image)

        image_filename = os.path.join(image_dir, f"image_{i+1:03d}.png")
        imsave(image_filename, image)

        theta = np.linspace(0.0, 360.0, max(image.shape), endpoint=False)
        sinogram = radon(image, theta=theta)

        noise_std = np.random.uniform(0, 13)
        sinogram += np.random.normal(0, noise_std, np.shape(sinogram))

        sinogram = normalize_image(sinogram)
        sinogram = img_as_ubyte(sinogram)
            
        sinogram_filename = os.path.join(image_dir, f"sinogram_{i+1:03d}.png")
        imsave(sinogram_filename, sinogram)

        inverse = iradon(sinogram, theta=theta)

        inverse = normalize_image(inverse)
        inverse = img_as_ubyte(inverse)

        inverse_filename = os.path.join(image_dir, f"inverse_{i+1:03d}.png")
        imsave(inverse_filename, inverse)

generate_images_and_sinograms(100)

C:\Users\Christos\AppData\Local\Temp\ipykernel_16216\1572917398.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  nb = int(np.random.uniform(1, 10, 1))
